Author: Jinze Wu

Student Number: p09323028

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

# **Download data**
Download data from google cloud

In [ ]:
!gdown --id '1oTtu7K5NVLCp-RKUuyVtwShLnryIVrKc' --output creditcard.xlsx

# **Sheet to DataFrame**

In [3]:
customer = pd.read_excel('creditcard.xlsx', sheet_name='客戶資料檔')
creditcard = pd.read_excel('creditcard.xlsx', sheet_name='信用卡資料檔')
creditlog = pd.read_excel('creditcard.xlsx', sheet_name='信用卡交易記錄檔')
creditlog = creditlog.iloc[:,:8]

In [4]:
customer.head()

,客戶ID,年齡,生日,居住地,教育程度,性別,婚姻狀況,職業,辦第一張信用卡的時間
0,7293,57,1955-08-24,大台北地區,專科,1:男,已婚,公務員,2006-11-27
1,5963,29,1983-12-05,大台北地區,學士,1:男,單身,白領一般職員,2009-07-24
2,18700,27,1985-12-20,高屏地區,高中,1:男,單身,藍領(勞動工作或作業員)一般職員,2009-09-22
3,920,39,1973-05-29,大台北地區,學士,1:男,已婚,自營事業老闆,2010-03-05
4,4612,26,1986-01-06,嘉南地區,學士,1:男,單身,白領一般職員,2010-08-05


In [5]:
creditcard.head()

,信用卡ID,客戶ID,信用卡開卡日,信用卡到期日,信用額度,卡等
0,41,2027,2006-05-19,2015-08-31,500000,金卡
1,814,7437,2002-01-14,2016-01-31,200000,普卡
2,905,6475,2002-05-07,2016-05-31,50000,普卡
3,1142,18841,2001-07-23,2015-07-31,250000,金卡
4,1411,14127,2005-10-03,2016-06-30,270000,白金卡


In [6]:
creditlog.head()

,交易序號,信用卡ID,客戶ID,刷卡日期,刷卡產品產業分類,刷卡類型,刷卡金額,刷卡地點
0,8,6448,15195,2010-12-01,14_旅遊,一般消費,5200,國內
1,9,6826,13687,2010-12-01,12_量販超市,一般消費,250,國內
2,10,6826,13687,2010-12-01,12_量販超市,一般消費,1316,國內
3,11,7896,179,2010-12-01,10_藥妝,一般消費,218,國內
4,20,9387,16756,2010-12-01,11_3C居家電信,一般消費,2610,國內


# **Clusters**

In [7]:
def classifier(df):
    if (df['性別'] == '1:男') and (df['婚姻狀況'] == '單身'):
        return 1
    if (df['性別'] == '1:男') and (df['婚姻狀況'] == '已婚'):
        return 2
    if (df['性別'] == '2:女') and (df['婚姻狀況'] == '單身'):
        return 3
    else: return 4

In [8]:
customer['cluster'] = customer.apply(classifier, axis=1)

In [9]:
customer

,客戶ID,年齡,生日,居住地,教育程度,性別,婚姻狀況,職業,辦第一張信用卡的時間,cluster
0,7293,57,1955-08-24,大台北地區,專科,1:男,已婚,公務員,2006-11-27,2
1,5963,29,1983-12-05,大台北地區,學士,1:男,單身,白領一般職員,2009-07-24,1
2,18700,27,1985-12-20,高屏地區,高中,1:男,單身,藍領(勞動工作或作業員)一般職員,2009-09-22,1
3,920,39,1973-05-29,大台北地區,學士,1:男,已婚,自營事業老闆,2010-03-05,2
4,4612,26,1986-01-06,嘉南地區,學士,1:男,單身,白領一般職員,2010-08-05,1
...,...,...,...,...,...,...,...,...,...,...
95,1611,64,1948-03-30,大台北地區,專科,2:女,已婚,白領主管,2003-09-17,4
96,11368,39,1973-02-06,台中地區,研究所以上,1:男,單身,白領一般職員,2001-11-26,1
97,867,47,1965-10-28,大台北地區,高中,1:男,已婚,白領主管,2002-10-11,2
98,17157,44,1968-02-16,高屏地區,專科,1:男,已婚,白領一般職員,2005-08-29,2


# **Individual Estimator**

In [10]:
IE = customer.drop_duplicates(subset=['客戶ID'], ignore_index=True)[['客戶ID']]

In [11]:
IE['mean_ie'] = IE['客戶ID'].map(creditlog.groupby(by=['客戶ID']).mean()['刷卡金額'])
IE['var_ie'] = IE['客戶ID'].map(creditlog.groupby(by=['客戶ID']).var()['刷卡金額'])

# **Group Estimator**

In [12]:
df = pd.merge(creditlog, customer, how='left', on='客戶ID')

In [13]:
GE = customer.drop_duplicates(subset=['cluster'], ignore_index=True)[['cluster']]

In [14]:
GE['mean_ge'] = GE['cluster'].map(df.groupby(by=['cluster']).mean()['刷卡金額'])
GE['var_ge'] = GE['cluster'].map(df.groupby(by=['cluster']).var()['刷卡金額'])

# **CRI**

In [31]:
CRI = customer.drop_duplicates(subset=['客戶ID'], ignore_index=True)[['客戶ID','cluster']]

In [32]:
CRI['刷卡次數'] = CRI['客戶ID'].map(creditlog['客戶ID'].value_counts())

In [33]:
CRI['mean_ie'] = CRI['客戶ID'].map(IE.groupby(by=['客戶ID']).mean()['mean_ie'])
CRI['var_ie'] = CRI['客戶ID'].map(IE.groupby(by=['客戶ID']).mean()['var_ie'])

CRI['mean_ge'] = CRI['cluster'].map(GE.groupby(by=['cluster']).mean()['mean_ge'])
CRI['var_ge'] = CRI['cluster'].map(GE.groupby(by=['cluster']).mean()['var_ge'])

In [40]:
CRI['be'] = (CRI.mean_ie*CRI.var_ge+CRI.mean_ge*CRI.var_ie/CRI['刷卡次數'])/(CRI.var_ge+CRI.var_ie/CRI['刷卡次數'])

In [41]:
CRI['CRI'] = (CRI.mean_ie - CRI.be) / (CRI.mean_ie - CRI.mean_ge) * 100

In [42]:
CRI.sort_values('客戶ID')

,客戶ID,cluster,刷卡次數,mean_ie,var_ie,mean_ge,var_ge,be,CRI
83,89,2,85,1772.717647,1.054061e+07,1656.062536,3.385888e+07,1772.291960,0.364911
73,106,2,75,1202.560000,6.690882e+06,1656.062536,3.385888e+07,1203.751754,0.262789
90,131,2,16,4347.375000,1.148968e+08,1656.062536,3.385888e+07,3876.456638,17.497722
80,179,2,4,2295.500000,2.542940e+06,1656.062536,3.385888e+07,2283.715182,1.842998
9,261,2,13,1524.230769,1.184193e+06,1656.062536,3.385888e+07,1524.584489,0.268312
...,...,...,...,...,...,...,...,...,...
24,19239,4,13,3127.230769,1.749050e+07,2688.464823,3.209252e+07,3109.576397,4.023642
72,19483,3,107,1735.224299,5.004884e+06,1426.744349,2.495288e+07,1734.647130,0.187101
50,19504,2,98,977.367347,1.478409e+06,1656.062536,3.385888e+07,977.669605,0.044535
91,19639,2,201,1626.716418,1.412979e+07,1656.062536,3.385888e+07,1626.777220,0.207189
